In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import sklearn
import keras

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

from Classfication import model
from Raw_Data_Read import extract_data

%matplotlib inline
seed = 5

# Input

In [ ]:
training_person = ['Player']
training_sensor = ['Hand','Palm','Right_Leg','Left_Leg']  

#testing_person = ['Player']
#testing_sensor = ['Hand','Palm','Right_Leg','Left_Leg']  

In [ ]:
training_data,training_label = extract_data(training_person,training_sensor)

#testing_data,testing_label = extract_data(testing_person,testing_sensor)

# One-Hot-Encoding for the Activity Labels

In [ ]:
training_data.shape

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_label)
#integer_encoded_testing = label_encoder.fit_transform(testing_label)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#integer_encoded_testing = integer_encoded_testing.reshape(len(integer_encoded_testing), 1)
training_label_encoded = onehot_encoder.fit_transform(integer_encoded)
#testing_label_encoded = onehot_encoder.fit_transform(integer_encoded_testing)

# Define Parameters 

In [ ]:
numOfRows = training_data.shape[1]
numOfColumns = training_data.shape[2]
numClasses = training_label_encoded.shape[1]


reshapedSegments = training_data.reshape(training_data.shape[0], numOfRows, numOfColumns)

X_train, X_test, y_train, y_test = train_test_split(reshapedSegments,training_label_encoded ,test_size=0.2, random_state=seed)

# Calling the Classfication Model

In [ ]:
model = model()
print(model.summary())
keras.utils.plot_model(model, show_shapes=True)

# Model Fit

In [ ]:
model = model()
history = model.fit(X_train,y_train,epochs=150,batch_size=400,validation_split=0.25)

In [ ]:
np.mean(history.history['acc'][140:150])

In [ ]:
np.mean(history.history['val_acc'][140:150])

In [ ]:
model = model()
history = model.fit(X_train,y_train,epochs=128,batch_size=300,validation_split=0.2)

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test, axis=1)
cm = metrics.confusion_matrix(max_y_test, max_y_pred_test)

In [ ]:
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = onehot_encoder.inverse_transform(y_test)
cm = metrics.confusion_matrix(max_y_test, max_y_pred_test)

In [ ]:
np.histogram(max_y_pred_test)

In [ ]:
y_test[0,:]

In [ ]:
print(accuracy_score(max_y_test, max_y_pred_test))

In [ ]:
history.history.keys()

# Plot the Results

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
#plt.ylim([0.5, 1])
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
fig.savefig('Model_Accuracy_Avijoy_Hand_Palm.jpg',dpi=300, bbox_inches='tight', pad_inches=0)


fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
#plt.ylim([0.5, 1])
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
fig.savefig('Training_Validation_Loss_Participant_Hand_Palm.jpg',dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
def plot_confusion_matrix (cm, classes, normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8,8))
    plt.grid(b=False)    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
target_names = ['Forehand Service','Backhand Service','Clear Lob Overhead Forehand','Clear Lob Overhead Backhand','Clear Lob Underarm Forehand',
         'Clear Lob Underarm Backhand','Net Shot Underarm Forehand','Net Shot Underarm Backhand','Drop Shot Overhead Forehand','Drop Shot Overhead Backhand',
         'Smash Overhead Forehand','Smash Overhead Backhand']

In [ ]:
cm_cmap=plt.cm.Greens
cm = metrics.confusion_matrix(max_y_test, max_y_pred_test)
plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(cm, classes=target_names, normalize=True, title='Normalized confusion matrix', cmap = cm_cmap)
plt.savefig('Confusion_Matrix.png', bbox_inches='tight', pad_inches=0,dpi=300)